In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from data import load_parquets_from_zip
import polars as pl
import numpy as np
import random
from torch.utils.data import TensorDataset, DataLoader
from tqdm.auto import tqdm
import os

In [3]:
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

In [4]:
imgs = load_parquets_from_zip('dataset/Ekstra_Bladet_image_embeddings.zip')['Ekstra_Bladet_image_embeddings/image_embeddings']

In [5]:
imgs_embs = torch.from_numpy(np.asarray(list(imgs['image_embedding'])))

In [6]:
dl = DataLoader(TensorDataset(imgs_embs), batch_size=512, shuffle=True)

In [7]:
class Normalize(nn.Module):
    
    def __init__(self):
        super().__init__()
        pass

    def forward(self, x):
        return F.normalize(x, dim=-1)

class EncodeDecoder(nn.Module):

    def __init__(self, in_size=1024, hidden=[512, 128]):
        super().__init__()
        layers = [in_size] + hidden
        self.encoder = nn.Sequential(*([nn.Linear(layers[i], layers[i+1], bias=False) for i in range(len(layers) - 1)] + [Normalize()]))
        layers = layers[::-1]
        self.decoder = nn.Sequential(*[nn.Linear(layers[i], layers[i+1], bias=False) for i in range(len(layers) - 1)])
        pass

    def forward(self, x):
        return self.decoder(self.encoder(x))

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = EncodeDecoder().to(device)

opt = torch.optim.Adam(model.parameters())

mse = nn.MSELoss()

In [9]:
model_path = 'preprocess/image_encoder.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
else:
    for e in range(1, 100):
        c_loss = 0
        c_step = 0
        with tqdm(dl, leave=False) as dll:
            for x in dll:
                opt.zero_grad()
                x = x[0].to(device)
                x_pred = model(x)
                loss = mse(x_pred, x)
                loss.backward()
                opt.step()
                c_loss += loss.item()
                c_step += 1
                dll.set_postfix(loss=c_loss / c_step)
        print(f'Epoch {e}: {c_loss / c_step }')
    torch.save(model.state_dict(), model_path)
model.eval()

  0%|          | 0/201 [00:00<?, ?it/s]

Epoch 1: 4.091748173857072e-05


  0%|          | 0/201 [00:00<?, ?it/s]

Epoch 2: 7.995236991116339e-07


  0%|          | 0/201 [00:00<?, ?it/s]

...

  0%|          | 0/201 [00:00<?, ?it/s]

Epoch 98: 8.286820037690844e-08


  0%|          | 0/201 [00:00<?, ?it/s]

Epoch 99: 6.589718893718922e-08


EncodeDecoder(
  (encoder): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=False)
    (1): Linear(in_features=512, out_features=128, bias=False)
    (2): Normalize()
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=False)
    (1): Linear(in_features=512, out_features=1024, bias=False)
  )
)

In [10]:
x = next(iter(dl))[0].to(device)

In [11]:
x

tensor([[-0.0325,  0.0286, -0.0039,  ...,  0.0428, -0.0180, -0.0141],
        [-0.0194, -0.0154, -0.0571,  ..., -0.0247,  0.0367,  0.0314],
        [-0.0080,  0.0134,  0.0466,  ...,  0.0630, -0.0346, -0.0435],
        ...,
        [-0.0051,  0.0427,  0.0274,  ...,  0.0329, -0.0334, -0.0161],
        [-0.0104,  0.0157,  0.0363,  ...,  0.0585, -0.0177, -0.0481],
        [-0.0461, -0.0466, -0.0732,  ..., -0.0009,  0.0293,  0.0025]],
       device='cuda:0')

In [12]:
model(x)

tensor([[-0.0324,  0.0286, -0.0039,  ...,  0.0429, -0.0181, -0.0143],
        [-0.0196, -0.0156, -0.0572,  ..., -0.0245,  0.0363,  0.0307],
        [-0.0077,  0.0134,  0.0467,  ...,  0.0633, -0.0348, -0.0433],
        ...,
        [-0.0050,  0.0430,  0.0274,  ...,  0.0330, -0.0339, -0.0162],
        [-0.0105,  0.0161,  0.0360,  ...,  0.0584, -0.0178, -0.0481],
        [-0.0465, -0.0460, -0.0730,  ..., -0.0010,  0.0293,  0.0022]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [13]:
torch.std(x, dim=0)

tensor([0.0186, 0.0231, 0.0307,  ..., 0.0282, 0.0268, 0.0226], device='cuda:0')

In [14]:
torch.mean(torch.abs(x - model(x)), dim=0)

tensor([0.0002, 0.0002, 0.0001,  ..., 0.0002, 0.0001, 0.0002], device='cuda:0',
       grad_fn=<MeanBackward1>)

In [15]:
torch.std(torch.abs(x - model(x)), dim=0)

tensor([0.0001, 0.0002, 0.0001,  ..., 0.0001, 0.0001, 0.0002], device='cuda:0',
       grad_fn=<StdBackward0>)

In [16]:
dl = DataLoader(TensorDataset(imgs_embs), batch_size=512, shuffle=False)

embs = []
with torch.no_grad():
    for x in dl:
        x = x[0].to(device)
        emb = model.encoder(x).cpu().numpy()
        embs.append(emb)

In [17]:
embs = np.concatenate(embs, axis=0)

In [18]:
embs

array([[ 0.11453101,  0.00137664, -0.01590084, ..., -0.02036735,
        -0.05598754,  0.0513007 ],
       [-0.0300234 ,  0.00979323, -0.00183117, ...,  0.0564303 ,
        -0.07377557,  0.09771489],
       [ 0.2675379 , -0.06965413, -0.18188432, ..., -0.1330114 ,
         0.00300317, -0.02472829],
       ...,
       [-0.05685404,  0.05275042,  0.06841292, ..., -0.02606313,
         0.0475052 ,  0.02816776],
       [ 0.0441104 ,  0.01659502, -0.03042564, ...,  0.06748616,
         0.01042796, -0.05028164],
       [-0.02042744,  0.08351305, -0.08589312, ...,  0.10827801,
        -0.06187802, -0.02863881]], dtype=float32)

In [19]:
embs = pl.DataFrame(data=embs, schema=['embeddings']).with_columns(imgs['article_id']).select('article_id', 'embeddings')

In [20]:
embs.write_parquet('preprocess/image_embs.parquet')